In [1]:
import json
import pandas as pd
from typing import Dict, List, Optional, Union
from pydantic import BaseModel, Field
from datasets import Dataset
from bespokelabs import curator
import argparse
from pathlib import Path
from typing import List, Dict
import litellm 
litellm.drop_params = True


import os
os.environ["CURATOR_VIEWER"] = "1"
os.environ["BESPOKE_API_KEY"] = "bespoke-9ff97e97445794907bec5ef8d57fd925da61e8ba7586c17ec2dca4bdffb87b9b"

/Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================================================
# Pydantic Models for Structured Outputs
# ============================================================================

class MolecularProperty(BaseModel):
    """Molecular property prediction with confidence and reasoning."""
    property_name: str = Field(description="Name of the molecular property")
    predicted_value: float = Field(description="Predicted numerical value")
    confidence: str = Field(description="Confidence level: high, medium, low")
    reasoning: str = Field(description="Chemical reasoning for the prediction")
    key_structural_features: List[str] = Field(description="Important structural features affecting the property")


In [3]:
# ============================================================================
# Core Dataset Generators
# ============================================================================
class PropertyPredictor(curator.LLM):
    """Generates property prediction tasks with reasoning."""
    
    response_format = MolecularProperty
    
    def prompt(self, input: Dict) -> str:
        smiles = input['canonical_smiles']
        property_type = input.get('standard_type', 'LogD')

        return f"""You are an expert medicinal chemistry research assistant. You have knowledge in areas including:

- Drug design and development - structure-activity relationships (SAR), lead optimization, pharmacophore modeling
- Synthetic chemistry - reaction mechanisms, synthetic routes, protecting group strategies
- Pharmacology - drug-target interactions, pharmacokinetics (ADME), pharmacodynamics
- Computational chemistry - molecular modeling, QSAR and ADME predictions
- Analytical techniques - NMR, MS, HPLC and other characterization methods
- Medicinal chemistry strategies - bioisosteres, prodrugs, fragment-based drug design

Analyze the following molecule and predict its {property_type}.

Molecule (SMILES): {smiles}

Provide a detailed prediction including:
1. The predicted {property_type} value
2. Your confidence level
3. Chemical reasoning based on structural features
4. Key structural features that influence this property

Consider factors like:
- Lipophilicity and hydrophilicity
- Molecular weight and size
- Functional groups present
- Aromatic systems
- Hydrogen bonding potential
- Charge distribution"""

    def parse(self, input: Dict, response: MolecularProperty) -> Dict:
        return {
            "smiles": input['canonical_smiles'],
            "actual_value": input.get('standard_value'),
            "predicted_value": response.predicted_value,
            "property_type": input.get('standard_type', 'LogD'),
            "confidence": response.confidence,
            "reasoning": response.reasoning,
            "key_features": response.key_structural_features,
        }

In [4]:
df = pd.read_csv('/Users/ldodda/Documents/Codes/GraphGen/test_logd.csv')

## Sampling from all bins of LogD Data to make sure I am getting a diverse set

In [5]:
# Create bins for logd values covering the full range -8 to +8
import numpy as np

# Create 16 bins from -8 to +8
bins = np.linspace(-8, 8, 17)  # 17 edges create 16 bins
df['logd_bin'] = pd.cut(df['logd'], bins=bins, include_lowest=True)

# Sample from each bin (e.g., 20 samples per bin)
samples_per_bin = 100
sampled_df = df.groupby('logd_bin', observed=False, group_keys=False).apply(
    lambda x: x.sample(min(len(x), samples_per_bin), random_state=42)
)

print(f"Original dataset size: {len(df)}")
print(f"Sampled dataset size: {len(sampled_df)}")
print("\nSamples per bin:")
print(sampled_df['logd_bin'].value_counts().sort_index())
print(f"\nBin width: {(8 - (-8))/16} = 1.0 LogD unit per bin")

Original dataset size: 9999
Sampled dataset size: 1075

Samples per bin:
logd_bin
(-8.001, -7.0]      2
(-7.0, -6.0]        4
(-6.0, -5.0]        4
(-5.0, -4.0]       12
(-4.0, -3.0]       38
(-3.0, -2.0]       68
(-2.0, -1.0]      100
(-1.0, 0.0]       100
(0.0, 1.0]        100
(1.0, 2.0]        100
(2.0, 3.0]        100
(3.0, 4.0]        100
(4.0, 5.0]        100
(5.0, 6.0]        100
(6.0, 7.0]        100
(7.0, 8.0]         47
Name: count, dtype: int64

Bin width: 1.0 = 1.0 LogD unit per bin


/var/folders/2d/s6tm3wj949lf8cdhpv06q97r0000gr/T/ipykernel_78072/2257146822.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('logd_bin', observed=False, group_keys=False).apply(


In [6]:
sampled_df.drop(columns=['logd_bin'], inplace=True)

In [7]:
sampled_df

,smiles,logd
5046,CC(C)CCOc1ccc[n+](CC(P(=O)([O-])O)P(=O)(O)O)c1,-7.810
2726,NC(Cc1ccc(O)cc1)C(=O)O,-7.260
1794,CC(C)CCCC(C)CCn1cc[n+](CC(P(=O)([O-])O)P(=O)(O...,-6.070
2746,CN(C)C(N)=NC(=N)N,-6.130
453,O=P(O)(O)C(CNc1nccs1)P(=O)(O)O,-6.350
...,...,...
993,N#Cc1nc(-c2ccccc2)oc1S(=O)(=O)c1ccc(Cl)cc1,7.120
7280,COc1ccc(CNc2nc(NCCOC(=O)CCCCCCCCc3cn(CCC(=O)N[...,7.612
1702,CC(C)Oc1ccc(-c2nnc(-c3ccnn3C)s2)cc1Cl,7.040
4508,COC(=O)c1cc(OC(=O)c2ccc(OC)c(O)c2)c(OC)cc1CCN1...,7.280


In [8]:
sampled_df.columns = ['canonical_smiles', 'standard_value']
sampled_df['standard_type'] = 'LogD'

In [9]:
mol_data_dict = sampled_df.to_dict(orient='records')

In [10]:
logd_dataset = Dataset.from_list(mol_data_dict)

In [11]:
logd_dataset

Dataset({
    features: ['canonical_smiles', 'standard_value', 'standard_type'],
    num_rows: 1075
})

In [12]:
predictor = PropertyPredictor(
        model_name="bedrock/us.amazon.nova-pro-v1:0",
        backend="litellm"
)

[11/03/25 01:45:52] INFO     Getting rate limits for model:                 ]8;id=15336;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/litellm_online_request_processor.py\litellm_online_request_processor.py]8;;\:]8;id=503491;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/litellm_online_request_processor.py#213\213]8;;\
                             bedrock/us.amazon.nova-pro-v1:0                                                       

[11/03/25 01:45:54] INFO     Test call headers: {}                          ]8;id=787421;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/litellm_online_request_processor.py\litellm_online_request_processor.py]8;;\:]8;id=615466;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/litellm_online_request_processor.py#199\199]8;;\

In [ ]:
print("Generating predictions...")
results = predictor(logd_dataset)


Generating predictions...


                    INFO     Authenticated with Curator API key                                        ]8;id=388058;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/client.py\client.py]8;;\:]8;id=871972;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/client.py#42\42]8;;\

                    INFO     Running LiteLLMOnlineRequestProcessor completions with   ]8;id=343347;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=870673;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#133\133]8;;\
                             model: bedrock/us.amazon.nova-pro-v1:0                                                

                    INFO     Preparing request file(s) in                             ]8;id=182727;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=17368;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#233\233]8;;\
                             /Users/ldodda/.cache/curator/7c699271da109df0                                         

[11/03/25 01:45:55] INFO     Wrote 1075 requests to                                   ]8;id=813076;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py\base_request_processor.py]8;;\:]8;id=380406;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/base_request_processor.py#315\315]8;;\
                             /Users/ldodda/.cache/curator/7c699271da109df0/requests_0                              
                             .jsonl.                                                                               

                    WARNING  No manual max_requests_per_minute set, and        ]8;id=810880;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py\base_online_request_processor.py]8;;\:]8;id=188236;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py#197\197]8;;\
                             headers based detection failed, using default                                         
                             value of 200                                                                          

                    WARNING  No manual max_tokens_per_minute set, and headers  ]8;id=552436;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py\base_online_request_processor.py]8;;\:]8;id=757041;file:///Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/bespokelabs/curator/request_processor/online/base_online_request_processor.py#216\216]8;;\
                             based detection failed, using default value of                                        
                             100000                                                                                

/Users/ldodda/Documents/Codes/GraphGen/.venv/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [ ]:
print("\n📊 Results:")
logD_dataset_df = results.dataset.to_pandas()

In [ ]:
logD_dataset_df

In [ ]:
results.dataset

In [ ]:
results.dataset.to_list()

In [ ]:
output_file = "logd_predictor.jsonl"
with open(output_file, 'w') as f:
    for item in results.dataset.to_list():
        f.write(json.dumps(item) + '\n')

In [ ]:
"""
Data Preparation Script for Llama 3.2 3B Fine-tuning
Converts bespokelabs-curator dataset to instruction-following format
"""

system_prompt = """You are an expert medicinal chemistry research assistant. You have knowledge in areas including:

- Drug design and development - structure-activity relationships (SAR), lead optimization, pharmacophore modeling
- Synthetic chemistry - reaction mechanisms, synthetic routes, protecting group strategies
- Pharmacology - drug-target interactions, pharmacokinetics (ADME), pharmacodynamics
- Computational chemistry - molecular modeling, QSAR and predicting ADME properties
- Analytical techniques - NMR, MS, HPLC and other characterization methods
- Medicinal chemistry strategies - bioisosteres, prodrugs, fragment-based drug design

You analyze molecular structures using SMILES notation and provide accurate predictions for LogD values (distribution coefficient at pH 7.4). 
Your predictions are based on careful analysis of structural features, lipophilic and hydrophilic contributions, and established cheminformatics principles.
"""
class DatasetConverter:
    """Convert molecular property dataset to instruction-following format."""
    
    def __init__(self, input_path: str, output_path: str):
        self.input_path = Path(input_path)
        self.output_path = Path(output_path)
        
        if not self.input_path.exists():
            raise FileNotFoundError(f"Input file not found: {input_path}")
    
    def convert_to_instruction_format(self) -> None:
        """Convert dataset to Llama 3.2 instruction format."""
        
        print(f"Reading dataset from: {self.input_path}")
        
        with open(self.input_path, 'r') as f_in:
            with open(self.output_path, 'w') as f_out:
                example_count = 0
                
                for line_num, line in enumerate(f_in, 1):
                    try:
                        example = json.loads(line)
                        converted = self._create_instruction_example(example)
                        f_out.write(json.dumps(converted) + '\n')
                        example_count += 1
                        
                    except json.JSONDecodeError as e:
                        print(f"Warning: Invalid JSON on line {line_num}: {e}")
                    except KeyError as e:
                        print(f"Warning: Missing field on line {line_num}: {e}")
        
        print(f"✓ Converted {example_count} examples")
        print(f"✓ Output saved to: {self.output_path}")
    
    @staticmethod
    def _create_instruction_example(entry: Dict) -> Dict:
        """
        Convert single entry to instruction format.
        
        Args:
            entry: Original entry with SMILES, values, and reasoning
            
        Returns:
            Dictionary with 'text' field containing formatted conversation
        """
        
        smiles = entry['smiles']
        actual_value = entry['actual_value']
        reasoning = entry['reasoning']
        key_features = ', '.join(entry['key_features'])
        
        # Create the full conversation in Llama 3.2 format
        text = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the following molecule and predict its LogD value.

SMILES: {smiles}

Key Structural Features: {key_features}

Chemical Analysis and Reasoning:
{reasoning}

Based on this analysis, what is the predicted LogD value for this molecule?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Based on the detailed structural analysis, considering the balance of lipophilic and hydrophilic features:

The predicted LogD value is: **{actual_value}**

This prediction reflects:
- The lipophilic contributions from the {key_features.split(',')[0] if ',' in key_features else 'aromatic and aliphatic components'}
- The hydrophilic contributions from polar functional groups
<|eot_id|>"""
        
        return {"text": text}
    
    def validate_output(self) -> bool:
        """Validate the converted dataset."""
        
        print("\nValidating dataset...")
        
        try:
            examples = []
            with open(self.output_path, 'r') as f:
                for i, line in enumerate(f):
                    ex = json.loads(line)
                    if 'text' not in ex:
                        raise ValueError(f"Line {i}: missing 'text' field")
                    examples.append(ex)
            
            print(f"✓ Valid JSONL format")
            print(f"✓ Number of examples: {len(examples)}")
            
            # Show statistics
            text_lengths = [len(ex['text']) for ex in examples]
            print(f"✓ Average text length: {sum(text_lengths) / len(text_lengths):.0f} chars")
            print(f"✓ Min length: {min(text_lengths)} chars")
            print(f"✓ Max length: {max(text_lengths)} chars")
            
            # Show sample
            print(f"\n--- Sample Example (first 500 chars) ---")
            print(examples[0]['text'][:500] + "...\n")
            
            return True
            
        except Exception as e:
            print(f"✗ Validation failed: {e}")
            return False

In [ ]:
converter = DatasetConverter('./logd_predictor.jsonl', './logd_formatted.jsonl')
converter.convert_to_instruction_format()

In [ ]:
from datasets import Dataset


def load_jsonl_data(file_path):
    """Load JSONL dataset."""
    texts = []
    with open(file_path, 'r') as f:
        for line in f:
            example = json.loads(line)
            texts.append(example['text'])
    return texts

DATASET_PATH = './logd_formatted.jsonl'
print(f"Loading dataset from {DATASET_PATH}...")
texts = load_jsonl_data(DATASET_PATH)
print(f"Loaded {len(texts)} examples")

# Create HF dataset
dataset = Dataset.from_dict({
    'text': texts
})

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
dataset

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr

import matplotlib.pyplot as plt

# Calculate regression metrics
actual_values = logD_dataset_df['actual_value']
predicted_values = logD_dataset_df['predicted_value']

# Regression metrics
mse = mean_squared_error(actual_values, predicted_values)
rmse = mse ** 0.5
mae = mean_absolute_error(actual_values, predicted_values)
r2 = r2_score(actual_values, predicted_values)

# Spearman correlation
spearman_corr, spearman_p = spearmanr(actual_values, predicted_values)

# Pearson correlation (for comparison)
pearson_corr = actual_values.corr(predicted_values)

print("📊 LogD Prediction Performance Metrics:")
print("=" * 50)
print(f"Mean Squared Error (MSE):     {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE):    {mae:.4f}")
print(f"R² Score:                     {r2:.4f}")
print(f"Pearson Correlation:          {pearson_corr:.4f}")
print(f"Spearman Correlation:         {spearman_corr:.4f}")
print(f"Spearman p-value:             {spearman_p:.2e}")

# Create scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(actual_values, predicted_values, alpha=0.6, s=50)
plt.plot([actual_values.min(), actual_values.max()], 
         [actual_values.min(), actual_values.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual LogD Values')
plt.ylabel('Predicted LogD Values')
plt.title(f'LogD Prediction Performance\nSpearman ρ = {spearman_corr:.3f}, R² = {r2:.3f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

# Create LogD zones classification
def classify_logd(value):
    if value < -2:
        return 'Low (<-2)'
    elif value <= 4:
        return 'Medium (-2 to 4)'
    else:
        return 'High (>4)'

# Apply classification to actual and predicted values
logD_dataset_df['actual_zone'] = actual_values.apply(classify_logd)
logD_dataset_df['predicted_zone'] = predicted_values.apply(classify_logd)

# Generate classification statistics

# Calculate classification metrics
accuracy = accuracy_score(logD_dataset_df['actual_zone'], logD_dataset_df['predicted_zone'])

print("📊 LogD Zone Classification Performance:")
print("=" * 60)
print(f"Overall Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(logD_dataset_df['actual_zone'], logD_dataset_df['predicted_zone']))

# Create confusion matrix
cm = confusion_matrix(logD_dataset_df['actual_zone'], logD_dataset_df['predicted_zone'], 
                     labels=['Low (<-2)', 'Medium (-2 to 4)', 'High (>4)'])

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Low (<-2)', 'Medium (-2 to 4)', 'High (>4)'],
            yticklabels=['Low (<-2)', 'Medium (-2 to 4)', 'High (>4)'])
plt.title('Confusion Matrix - LogD Zone Classification')
plt.xlabel('Predicted Zone')
plt.ylabel('Actual Zone')
plt.show()

# Show distribution of zones
print("\n📈 Zone Distribution:")
print("Actual zones:")
print(logD_dataset_df['actual_zone'].value_counts().sort_index())
print("\nPredicted zones:")
print(logD_dataset_df['predicted_zone'].value_counts().sort_index())

In [ ]:
#dataset.push_to_hub("lsdodda/LogD-Predictor")